In [1]:
# Data manipulation
import pandas as pd
import numpy as np

# Graphics
import matplotlib.pyplot as plt
import seaborn as sns

# Modeling
import lightgbm as lgb
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import f1_score, recall_score, accuracy_score, precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix

# Divers
from time import time, strftime, gmtime
import gc
import pickle

# Module des fonctions du notebook
import fonctions08 as f

# Autoreload pour prise en compte des changments dans le module fonctions
%load_ext autoreload
%autoreload 1
%aimport fonctions08

In [2]:
# Heure démarrage
t0=time()

# Constantes
RANDOM_STATE = 1
NROWS = None
NCOLS = None
N_FOLDS = 5

# Paramètres pour les validations croisées
random_seed = 1 # Seed pour générateur de nombre aléatoire
folds = 5 # Nombre de folds pour validation croisée

In [3]:
features = f.import_csv("data/features/train_features_selected.csv", nrows=NROWS)
target = f.import_csv("data/features/train_target.csv", nrows=NROWS)

features = features.set_index("SK_ID_CURR")
target = target.set_index("SK_ID_CURR")

print(features.shape)
print(target.shape)

Memory usage of dataframe is 505.82 MB
Memory usage after optimization is: 164.84 MB
Decreased by 67.4%
Memory usage of dataframe is 3.28 MB
Memory usage after optimization is: 1.03 MB
Decreased by 68.7%
(215257, 307)
(215257, 1)


# Echantillon avec classes équilibrées

In [4]:
target[target['TARGET'] == 1].shape

(17377, 1)

In [5]:
target_0 = target[target['TARGET'] == 0]
target_1 = target[target['TARGET'] == 1]

target_0_sample = target_0.sample(len(target_1), random_state=RANDOM_STATE)

target_sample_balanced = pd.concat([target_0_sample, target_1], axis=0)
features_sample_balanced = features.loc[target_sample_balanced.index, :].iloc[:, :NCOLS]

In [6]:
features_sample_balanced.shape

(34754, 307)

In [7]:
# Creation d'un train set de 10000 observations et test set de 6000 obs
X_train, X_test, y_train, y_test =\
    train_test_split(features_sample_balanced, target_sample_balanced,
                     train_size=15000, test_size=6000,
                     stratify=target_sample_balanced, random_state=RANDOM_STATE)

# TEST CV + LGBM + F1 SCORE

In [8]:
import optuna

def objective(trial):
    params = {
        'verbosity':-1,
        'objective': 'binary',
        'metric': 'auc',        
        'boosting_type': 'gbdt',
        'n_estimators': trial.suggest_int('n_estimators', 20, 200),
        'num_leaves ': trial.suggest_int('num_leaves', 8, 128),
        'max_depth': trial.suggest_int('max_depth', 4, 10),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.1),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_uniform('reg_alpha', 0.0, 1.0),
    }
    
    k_fold = KFold(n_splits=N_FOLDS, shuffle=False, random_state=RANDOM_STATE)
    
    scores=[]

    for train_index, valid_index in k_fold.split(X_train):
    
        # Training data for the fold
        fold_X_train, fold_y_train = X_train.iloc[train_index, :], y_train.iloc[train_index, :]
        
        # Validation data for the fold
        fold_X_valid, fold_y_valid = X_train.iloc[valid_index, :], y_train.iloc[valid_index, :]
        
        model = lgb.LGBMModel(random_state=1)
        model.set_params(**params)
        model.fit(fold_X_train, np.ravel(fold_y_train))
        preds = model.predict(fold_X_valid)
        scores.append(accuracy_score(fold_y_valid, np.rint(preds)))
   
    return np.mean(scores)

In [9]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000, timeout=None) # timeout (seconds)

[I 2020-01-04 12:07:12,968] Finished trial#0 resulted in value: 0.6840666666666666. Current best value is 0.6840666666666666 with parameters: {'n_estimators': 34, 'num_leaves': 53, 'max_depth': 7, 'learning_rate': 0.03570944980195855, 'min_child_samples': 34, 'colsample_bytree': 0.9567465956112252, 'reg_alpha': 0.8143452695877855}.
[I 2020-01-04 12:07:19,422] Finished trial#1 resulted in value: 0.6959333333333333. Current best value is 0.6959333333333333 with parameters: {'n_estimators': 68, 'num_leaves': 92, 'max_depth': 5, 'learning_rate': 0.0474360122439793, 'min_child_samples': 65, 'colsample_bytree': 0.8236928674965407, 'reg_alpha': 0.8432969554380936}.
[I 2020-01-04 12:07:28,382] Finished trial#2 resulted in value: 0.6994. Current best value is 0.6994 with parameters: {'n_estimators': 150, 'num_leaves': 27, 'max_depth': 7, 'learning_rate': 0.09787444090695148, 'min_child_samples': 94, 'colsample_bytree': 0.5022079525942565, 'reg_alpha': 0.2326000947889294}.
[I 2020-01-04 12:07:40

[I 2020-01-04 12:11:35,729] Finished trial#24 resulted in value: 0.7017333333333333. Current best value is 0.7035333333333333 with parameters: {'n_estimators': 134, 'num_leaves': 38, 'max_depth': 8, 'learning_rate': 0.0948815265863441, 'min_child_samples': 27, 'colsample_bytree': 0.8598282874151197, 'reg_alpha': 0.25771303205529483}.
[I 2020-01-04 12:11:51,493] Finished trial#25 resulted in value: 0.7003333333333334. Current best value is 0.7035333333333333 with parameters: {'n_estimators': 134, 'num_leaves': 38, 'max_depth': 8, 'learning_rate': 0.0948815265863441, 'min_child_samples': 27, 'colsample_bytree': 0.8598282874151197, 'reg_alpha': 0.25771303205529483}.
[I 2020-01-04 12:12:06,354] Finished trial#26 resulted in value: 0.6992666666666667. Current best value is 0.7035333333333333 with parameters: {'n_estimators': 134, 'num_leaves': 38, 'max_depth': 8, 'learning_rate': 0.0948815265863441, 'min_child_samples': 27, 'colsample_bytree': 0.8598282874151197, 'reg_alpha': 0.257713032055

[I 2020-01-04 12:18:44,890] Finished trial#48 resulted in value: 0.6980666666666667. Current best value is 0.7046666666666667 with parameters: {'n_estimators': 133, 'num_leaves': 51, 'max_depth': 9, 'learning_rate': 0.0659897881452932, 'min_child_samples': 35, 'colsample_bytree': 0.9672271945239522, 'reg_alpha': 0.9782469366535855}.
[I 2020-01-04 12:19:03,720] Finished trial#49 resulted in value: 0.7019333333333334. Current best value is 0.7046666666666667 with parameters: {'n_estimators': 133, 'num_leaves': 51, 'max_depth': 9, 'learning_rate': 0.0659897881452932, 'min_child_samples': 35, 'colsample_bytree': 0.9672271945239522, 'reg_alpha': 0.9782469366535855}.
[I 2020-01-04 12:19:19,939] Finished trial#50 resulted in value: 0.7002. Current best value is 0.7046666666666667 with parameters: {'n_estimators': 133, 'num_leaves': 51, 'max_depth': 9, 'learning_rate': 0.0659897881452932, 'min_child_samples': 35, 'colsample_bytree': 0.9672271945239522, 'reg_alpha': 0.9782469366535855}.
[I 2020

[I 2020-01-04 12:25:23,637] Finished trial#72 resulted in value: 0.7012. Current best value is 0.7046666666666667 with parameters: {'n_estimators': 133, 'num_leaves': 51, 'max_depth': 9, 'learning_rate': 0.0659897881452932, 'min_child_samples': 35, 'colsample_bytree': 0.9672271945239522, 'reg_alpha': 0.9782469366535855}.
[I 2020-01-04 12:25:39,583] Finished trial#73 resulted in value: 0.7020000000000001. Current best value is 0.7046666666666667 with parameters: {'n_estimators': 133, 'num_leaves': 51, 'max_depth': 9, 'learning_rate': 0.0659897881452932, 'min_child_samples': 35, 'colsample_bytree': 0.9672271945239522, 'reg_alpha': 0.9782469366535855}.
[I 2020-01-04 12:25:57,908] Finished trial#74 resulted in value: 0.7007999999999999. Current best value is 0.7046666666666667 with parameters: {'n_estimators': 133, 'num_leaves': 51, 'max_depth': 9, 'learning_rate': 0.0659897881452932, 'min_child_samples': 35, 'colsample_bytree': 0.9672271945239522, 'reg_alpha': 0.9782469366535855}.
[I 2020

[I 2020-01-04 12:32:30,989] Finished trial#96 resulted in value: 0.7027333333333333. Current best value is 0.7048666666666665 with parameters: {'n_estimators': 165, 'num_leaves': 42, 'max_depth': 9, 'learning_rate': 0.08271076062276324, 'min_child_samples': 27, 'colsample_bytree': 0.9986055731715608, 'reg_alpha': 0.9382799792144039}.
[I 2020-01-04 12:32:50,798] Finished trial#97 resulted in value: 0.7002. Current best value is 0.7048666666666665 with parameters: {'n_estimators': 165, 'num_leaves': 42, 'max_depth': 9, 'learning_rate': 0.08271076062276324, 'min_child_samples': 27, 'colsample_bytree': 0.9986055731715608, 'reg_alpha': 0.9382799792144039}.
[I 2020-01-04 12:33:09,024] Finished trial#98 resulted in value: 0.6982666666666666. Current best value is 0.7048666666666665 with parameters: {'n_estimators': 165, 'num_leaves': 42, 'max_depth': 9, 'learning_rate': 0.08271076062276324, 'min_child_samples': 27, 'colsample_bytree': 0.9986055731715608, 'reg_alpha': 0.9382799792144039}.
[I 2

...

[I 2020-01-04 15:51:56,916] Finished trial#895 resulted in value: 0.7049333333333333. Current best value is 0.7088 with parameters: {'n_estimators': 196, 'num_leaves': 113, 'max_depth': 9, 'learning_rate': 0.0547539016948516, 'min_child_samples': 63, 'colsample_bytree': 0.5117860715863971, 'reg_alpha': 0.9655472592107446}.
[I 2020-01-04 15:52:10,866] Finished trial#896 resulted in value: 0.7026. Current best value is 0.7088 with parameters: {'n_estimators': 196, 'num_leaves': 113, 'max_depth': 9, 'learning_rate': 0.0547539016948516, 'min_child_samples': 63, 'colsample_bytree': 0.5117860715863971, 'reg_alpha': 0.9655472592107446}.
[I 2020-01-04 15:52:25,254] Finished trial#897 resulted in value: 0.7062666666666667. Current best value is 0.7088 with parameters: {'n_estimators': 196, 'num_leaves': 113, 'max_depth': 9, 'learning_rate': 0.0547539016948516, 'min_child_samples': 63, 'colsample_bytree': 0.5117860715863971, 'reg_alpha': 0.9655472592107446}.
[I 2020-01-04 15:52:39,818] Finished 

[I 2020-01-04 15:58:00,656] Finished trial#920 resulted in value: 0.7065333333333333. Current best value is 0.7088 with parameters: {'n_estimators': 196, 'num_leaves': 113, 'max_depth': 9, 'learning_rate': 0.0547539016948516, 'min_child_samples': 63, 'colsample_bytree': 0.5117860715863971, 'reg_alpha': 0.9655472592107446}.
[I 2020-01-04 15:58:16,153] Finished trial#921 resulted in value: 0.7056. Current best value is 0.7088 with parameters: {'n_estimators': 196, 'num_leaves': 113, 'max_depth': 9, 'learning_rate': 0.0547539016948516, 'min_child_samples': 63, 'colsample_bytree': 0.5117860715863971, 'reg_alpha': 0.9655472592107446}.
[I 2020-01-04 15:58:31,210] Finished trial#922 resulted in value: 0.7036. Current best value is 0.7088 with parameters: {'n_estimators': 196, 'num_leaves': 113, 'max_depth': 9, 'learning_rate': 0.0547539016948516, 'min_child_samples': 63, 'colsample_bytree': 0.5117860715863971, 'reg_alpha': 0.9655472592107446}.
[I 2020-01-04 15:58:45,301] Finished trial#923 re

[I 2020-01-04 16:04:17,341] Finished trial#945 resulted in value: 0.7066666666666667. Current best value is 0.7088 with parameters: {'n_estimators': 196, 'num_leaves': 113, 'max_depth': 9, 'learning_rate': 0.0547539016948516, 'min_child_samples': 63, 'colsample_bytree': 0.5117860715863971, 'reg_alpha': 0.9655472592107446}.
[I 2020-01-04 16:04:32,521] Finished trial#946 resulted in value: 0.7031333333333334. Current best value is 0.7088 with parameters: {'n_estimators': 196, 'num_leaves': 113, 'max_depth': 9, 'learning_rate': 0.0547539016948516, 'min_child_samples': 63, 'colsample_bytree': 0.5117860715863971, 'reg_alpha': 0.9655472592107446}.
[I 2020-01-04 16:04:48,457] Finished trial#947 resulted in value: 0.7038666666666666. Current best value is 0.7088 with parameters: {'n_estimators': 196, 'num_leaves': 113, 'max_depth': 9, 'learning_rate': 0.0547539016948516, 'min_child_samples': 63, 'colsample_bytree': 0.5117860715863971, 'reg_alpha': 0.9655472592107446}.
[I 2020-01-04 16:05:03,41

[I 2020-01-04 16:10:32,625] Finished trial#970 resulted in value: 0.7064666666666667. Current best value is 0.7088 with parameters: {'n_estimators': 196, 'num_leaves': 113, 'max_depth': 9, 'learning_rate': 0.0547539016948516, 'min_child_samples': 63, 'colsample_bytree': 0.5117860715863971, 'reg_alpha': 0.9655472592107446}.
[I 2020-01-04 16:10:47,636] Finished trial#971 resulted in value: 0.7040000000000001. Current best value is 0.7088 with parameters: {'n_estimators': 196, 'num_leaves': 113, 'max_depth': 9, 'learning_rate': 0.0547539016948516, 'min_child_samples': 63, 'colsample_bytree': 0.5117860715863971, 'reg_alpha': 0.9655472592107446}.
[I 2020-01-04 16:11:07,005] Finished trial#972 resulted in value: 0.7040666666666666. Current best value is 0.7088 with parameters: {'n_estimators': 196, 'num_leaves': 113, 'max_depth': 9, 'learning_rate': 0.0547539016948516, 'min_child_samples': 63, 'colsample_bytree': 0.5117860715863971, 'reg_alpha': 0.9655472592107446}.
[I 2020-01-04 16:11:22,06

[I 2020-01-04 16:17:03,749] Finished trial#995 resulted in value: 0.7048. Current best value is 0.7088 with parameters: {'n_estimators': 196, 'num_leaves': 113, 'max_depth': 9, 'learning_rate': 0.0547539016948516, 'min_child_samples': 63, 'colsample_bytree': 0.5117860715863971, 'reg_alpha': 0.9655472592107446}.
[I 2020-01-04 16:17:18,826] Finished trial#996 resulted in value: 0.7030666666666667. Current best value is 0.7088 with parameters: {'n_estimators': 196, 'num_leaves': 113, 'max_depth': 9, 'learning_rate': 0.0547539016948516, 'min_child_samples': 63, 'colsample_bytree': 0.5117860715863971, 'reg_alpha': 0.9655472592107446}.
[I 2020-01-04 16:17:34,074] Finished trial#997 resulted in value: 0.7044666666666667. Current best value is 0.7088 with parameters: {'n_estimators': 196, 'num_leaves': 113, 'max_depth': 9, 'learning_rate': 0.0547539016948516, 'min_child_samples': 63, 'colsample_bytree': 0.5117860715863971, 'reg_alpha': 0.9655472592107446}.
[I 2020-01-04 16:17:50,522] Finished 

In [10]:
print('Number of finished trials: {}'.format(len(study.trials)))

print('Best trial:')
trial = study.best_trial

print('  Value: {}'.format(trial.value))

print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

Number of finished trials: 1000
Best trial:
  Value: 0.7088
  Params: 
    n_estimators: 196
    num_leaves: 113
    max_depth: 9
    learning_rate: 0.0547539016948516
    min_child_samples: 63
    colsample_bytree: 0.5117860715863971
    reg_alpha: 0.9655472592107446


In [11]:
# Modele final
model = lgb.LGBMModel(objective='binary', metric='auc',
                      boosting_type= 'gbdt')
model.set_params(**study.best_params)
model

LGBMModel(boosting_type='gbdt', class_weight=None,
     colsample_bytree=0.5117860715863971, importance_type='split',
     learning_rate=0.0547539016948516, max_depth=9, metric='auc',
     min_child_samples=63, min_child_weight=0.001, min_split_gain=0.0,
     n_estimators=196, n_jobs=-1, num_leaves=113, objective='binary',
     random_state=None, reg_alpha=0.9655472592107446, reg_lambda=0.0,
     silent=True, subsample=1.0, subsample_for_bin=200000,
     subsample_freq=0)

In [12]:
# On sauvegarde le modèle et la liste des features
model_params = {'model': model.get_params(), 'study': study}
filename = 'data/model/params_accuracy.sav'
pickle.dump(model_params, open(filename, 'wb'))

In [13]:
# Aperçu du dataframe des combinaisons de valeurs testées
df_res = pickle.load(open(filename, 'rb'))['study'].trials_dataframe()
df_res.head()

number                state     value             datetime_start  \
                                                                     
0      0  TrialState.COMPLETE  0.684067 2020-01-04 12:07:06.593023   
1      1  TrialState.COMPLETE  0.695933 2020-01-04 12:07:12.968129   
2      2  TrialState.COMPLETE  0.699400 2020-01-04 12:07:19.422601   
3      3  TrialState.COMPLETE  0.701333 2020-01-04 12:07:28.382430   
4      4  TrialState.COMPLETE  0.697600 2020-01-04 12:07:40.048824   

           datetime_complete           params                          \
                             colsample_bytree learning_rate max_depth   
0 2020-01-04 12:07:12.968129         0.956747      0.035709         7   
1 2020-01-04 12:07:19.422601         0.823693      0.047436         5   
2 2020-01-04 12:07:28.382430         0.502208      0.097874         7   
3 2020-01-04 12:07:40.048824         0.795568      0.054641         7   
4 2020-01-04 12:07:46.947284         0.768337      0.057053         5   

                                                      system_attrs  
  min_child_samples n_estimators num_leaves reg_alpha      _number  
0                34           34         53  0.814345            0  
1                65           68         92  0.843297            1  
2                94          150         27  0.232600            2  
3                89          119        124  0.651748            3  
4                33           79        100  0.345527            4

In [14]:
t1 = time()
print("computing time : {:8.6f} sec".format(t1-t0))
print("computing time : " + strftime('%H:%M:%S', gmtime(t1-t0)))

computing time : 15112.349312 sec
computing time : 04:11:52
